# Access data from Neuroverse SQL data stores

This notebook demonstrates the ability to access the data in a notebook.
<br><br>
Neuroverse organises it's sql data stores in Schema Manager. Schema Manager can be controlled through the UI or in a Notebook as shown in SchemaManagerCommands example notebook.
<br><br>
Data is ingested into the data stores using Data Stream Manager.
<br><br>
This data can be accessed either through Analysor (dashboards) or in Notebooks.


## Query objects
#### NeuroData.SqlQuery
     - The query object uses the constructor NeuroData.SqlQuery(). The constructor takes the following named parameters: select, tablename, subquery, alias, joins, where, groupby, having and orderby. Examples of are shown below
#### NeuroData.SqlJoin
     - The join object uses the constructor NeuroData.SqlJoin(). The constructor takes the following named parameters: jointype, tablename, subquery, alias and clause. Examples of are shown below

## Query functions
#### NeuroData.sqltodf
     - The function is executed as NeuroData.sqltodf(). It takes the named parameter sqlquery and storename. The function executes a query and returns a dataframe. Examples are shown below
#### NeuroData.sqltocsv
     - The function is executed as NeuroData.sqltocsv(). It takes the named parameters: folderpath, filename, sqlquery and storename. The function executes a query and returns a csv. Examples are shown below

## DataFrames
Dataframes are a tabular in memory data structure. Notebooks is capable of displaying formatted versions of the Dataframes. Documentation for dataframes can be found at [Link](http://juliadata.github.io/DataFrames.jl/stable/)

# Examples

## Load libraries

In [ ]:
using NeuroJulia
using DataFrames

## Query and subquery examples

Table1

Id | Date | Tag | Value
--- | --- | --- | ---
1 | 2017-10-10 08:40:52 | 3 | 4.4
1 | 2017-10-10 08:44:52 | 2 | 204.4
2 | 2017-10-10 08:43:52 | 1 | 78.4
3 | 2017-10-10 08:41:52 | 2 | 384.4

In [ ]:
tablename="Table1"
storename="Default"

In [ ]:
# select * from test where Id=1 order by Date
query1=NeuroData.SqlQuery(select="top 100 *",tablename=tablename,where="Tag!='1'")
#load into memory
df1=NeuroData.sqltodf(sqlquery=query1,storename=storename)
#create csv in current folder
NeuroData.sqltocsv(filename="result1.csv",sqlquery=query1,storename=storename)
#create csv in parent of current folder
NeuroData.sqltocsv(filename="../result2.csv",sqlquery=query1,storename=storename)
#create csv in notebooks 00_NeuroTemplates folder (00_NeuroTemplates is in the root directory)
NeuroData.sqltocsv(folderpathfromroot="/00_NeuroTemplates",filename="result3.csv",sqlquery=query1,storename=storename)
# Subquery
query2=NeuroData.SqlQuery(select="Top 100 q1.Date,q1.Value",subquery=query1,alias="q1",orderby="q1.Date")
#load into memory
df2=NeuroData.sqltodf(sqlquery=query2,storename=storename)

## Join example

In [ ]:
#This query pivots row based data into columns based on Tag and Value fields
query3=NeuroData.SqlQuery(select="Id,Date,Value",tablename=tablename,where="Tag=1")
query4=NeuroData.SqlQuery(select="Id,Date,Value",tablename=tablename,where="Tag=2")
join1=NeuroData.SqlJoin(jointype="INNER",subquery=query4,alias="t2",clause="t1.Id=t2.Id and t1.Date=t2.Date")
query5=NeuroData.SqlQuery(select="Id,Date,Value",tablename=tablename,where="Tag=3")
join2=NeuroData.SqlJoin(jointype="INNER",subquery=query5,alias="t3",clause="t1.Id=t3.Id and t1.Date=t3.Date")
query6=NeuroData.SqlQuery(select="t1.Id,t1.Date,t1.Value As Tag1,t2.Value As Tag2,t3.Value As Tag3",subquery=query3,alias="t1",joins=[join1,join2])
#load into memory
df3=NeuroData.sqltodf(sqlquery=query6,storename=storename)

## Query the csv
This shows how to query the csv created by sqltocsv and load the result into memory

This example uses the [Query](http://www.david-anthoff.com/Query.jl/stable/) and [csv](http://juliadata.github.io/CSV.jl/stable/) packages. 

In [ ]:
Pkg.add("Query")
Pkg.add("CSV")
using Query
using CSV

In [ ]:
q = @from i in CSV.Source("query1.csv",header=["Id","Date","Tag","Value"],datarow=2) begin
    @where i.Tag == "1" || i.Tag=="2"
    @select i
    @collect DataFrame
end

head(q)